In [1]:
from math import log

Autor. @LuisFalva
## La impureza de Gini es una magnitud de incertidumbre, esto se refiere a la aleatoriedad con la que se elige un _elemento$_{i}$_ del conjunto de datos y la probabilidad de asignarle al _elemento$_{i}$_ una etiqueta _m_ de forma incorrecta.

# $Gini = 1 - \sum_{i=1}^{c} (p_{i})^{2}$

## Análisis de la función.
### Para cada clase se deberá calcular la probabilidad que le corresponda, esta medida se le considera la probabilidad que posee cada clase de ser escogida. $p_{i}$ es la probabilidad calculada para cada clase, las probabilidades se pueden calcular por separado. Simplificando, podemos decir que crearemos primero (y por separado) una función que calcule las $p_{i}$'s y así poder usarla almacenando en un diccionario de python los valores de las $k$ clases.

### Supongamos que partimos la función y almacenamos en una nueva variable $P = \sum (p_{i})^2$ la suma las probabilidades al cuadrado. Ahora bien, si se aplica $1 - P$ esto representa el _complemento_ de la probabilidad $P$ i.e. la parte _contraria de la probabilidad_.

### Por lo tanto $1 - P$ la consideramos una maginitud contraria a la suma de las probabilidades al cuadrado $P$, esa magnitud describe la probabilidad de ser etiquetado de forma incorrecta por el nivel de impureza en los datos, de aquí nace la idea de la _magnitud de incertidumbre_, i.e. _Gini_.

### Por ahora crearemos nuestros nodos los cuales almacenarán información de cada clase

In [3]:
Root_node = {"Manzana": 100, "Platano": 400, "Mango": 5, "Uva": 550}
child_node_st = {"Manzana": 100, "Platano": 400, "Mango": 5}
child_node_nd = {"Platano": 400, "Mango": 5}
child_node_rd = {"Platano": 400}

In [4]:
# Ejercicio Dummy.
print("Datos de entrada:", Root_node)
print("Suma Total:", 100+400+5+550)
print("Prob Manzana:", 100/1055)
print("Prob Platano:", 400/1055)
print("Prob Mango:", 5/1055)
print("Prob Uva:", 550/1055)

# Codifiquemos...
node_sum = sum(Root_node.values())
dict_prob = {c:v / node_sum for c, v in Root_node.items()}
print("\nSuma de k probabilidades:", sum(dict_prob.values()))
print("Diccionario:")
dict_prob

Datos de entrada: {'Manzana': 100, 'Platano': 400, 'Mango': 5, 'Uva': 550}
Suma Total: 1055
Prob Manzana: 0.0947867298578199
Prob Platano: 0.3791469194312796
Prob Mango: 0.004739336492890996
Prob Uva: 0.5213270142180095

Suma de k probabilidades: 1.0
Diccionario:


{'Manzana': 0.0947867298578199,
 'Platano': 0.3791469194312796,
 'Mango': 0.004739336492890996,
 'Uva': 0.5213270142180095}

In [5]:
def probability_class(node):
    node_sum = sum(node.values())
    percents = {c:v / node_sum for c, v in node.items()}
    return node_sum, percents

## Gini es la medida que menos costo computacional requiere por las operaciones aritméticas que realiza la función (la suma de las probabilidades al cuadrado). Por ende, a las probabilidades menores casi no las toma en cuenta, y por ende tienden a influir muy poco (o casi nada) en el resultado final.

### _Cuenta demostrativa:_
#### $0.0948^{2} + 0.3791^{2} + 0.0047^{2} + 0.5213^{2}$
#### $0.009 + 0.1438 + 0.00002209 + 0.2718$
#### $P = 0.4246$
#### $Gini = 1 - P = 0.5754$

### La función _gini score_ es una función que nos ayudará a obtener la magnitud Gini (explicada al inicio del notebook), para ello debemos crear por agregación al método, la función _probability class_ para mejorar y atomizar ambos comportamientos.

In [6]:
def gini_score(node):
    _, percents = probability_class(node)
    # donde i contiene la probabilidad calculada del nodo en cuestión, con def "probability_class".
    score = round(1 - sum([i**2 for i in percents.values()]), 3)
    print('Gini Score for node {} : {}'.format(node, score))
    return score

### La función de Entropía es menos eficiente en términos computacionales por lo que realiza el algoritmo de $\log_{2}$, pero lo interesante aquí es que como usa el $\log$, a las probabilidades menores y mayores las ajusta y toma por *"equivalente"* aquellas probabilidades más pequeñas. Por lo que el resultado será mucho más sensible a dichas probabilidades, en los casos que se busque darle la misma relevancia a las probilidades más bajas, se recomienda usar la Entropía como medida de impureza.

# $E(S) = \sum_{i = 1}^{c} - p_{i} \log_{2} p_{i}$

### _Cuenta demostrativa:_
#### $- 0.0948 \log_{2}(0.0948) + -0.3791 \log_{2}(0.3791) + -0.0047 \log_{2}(0.0047) + -0.5213 \log_{2}(0.5213)$
#### $0.3222 + 0.5304 + 0.0363 + 0.4899$
#### $P = 1.3788$
#### $Gini = 1 - P = -0.3788$

In [7]:
def entropy_score(node):
    _, percents = probability_class(node)
    # donde i contiene la probabilidad calculada del nodo en cuestión, con def "probability_class".
    score = round(sum([-i * log(i, 2) for i in percents.values()]), 3)
    print('Entropy Score for node {} : {}'.format(node, score))
    return score

### Calculamos el information Gain, a menor impureza, mayor information Gain.

In [8]:
def information_gain(parent, children, criterion):
    score = {'gini': gini_score, 'entropy': entropy_score}
    metric = score[criterion]
    parent_score = metric(parent)
    parent_sum = sum(parent.values())
    weighted_child_score = sum([metric(i) * sum(i.values()) / parent_sum  for i in children])
    gain = round((parent_score - weighted_child_score),2)
    print('Information gain: {}'.format(gain))
    return gain

### Aquí se muestra el funcionamiento de cada función tanto gini como entropía, se podrá distinguir que si, intencionalmente, incluímos la categoría "Mango". El número 5 quiere decir que estamos incluyendo solamente 5 registros que contienen la clase "Mango", por lo tanto podemos notar que hay un importante desbalanceo entre las cantidades por cada clase, por ejemplo:
- En el caso de Mazana, Platano y Uva, éstas contienen cantidades por encima de los 100 registros, claramente son las clases predominantes en este ejemplo, por lo que serán de gran influencia en la separación de la información en cada nodo hijo
- En el caso de la clase Mango, podemos asumir que la probabilidad calculada para ella será muy pequeña, por lo que será muy probable que la función de magnitud Gini ignore dicha clase
- Por otro lado vemos que la función de Entropia, no solo logra distinguir a la clase minoritaria, sino que separa a mayor precisión la información que reside en cada clase dando como resultado final un mayor *Information Gain*

In [9]:
gini_gain = information_gain(parent=Root_node, children=[child_node_st, child_node_nd, child_node_rd], criterion='gini')

Gini Score for node {'Manzana': 100, 'Platano': 400, 'Mango': 5, 'Uva': 550} : 0.575
Gini Score for node {'Manzana': 100, 'Platano': 400, 'Mango': 5} : 0.333
Gini Score for node {'Platano': 400, 'Mango': 5} : 0.024
Gini Score for node {'Platano': 400} : 0.0
Information gain: 0.41


In [10]:
entropy_gain = information_gain(parent=Root_node, children=[child_node_st, child_node_nd, child_node_rd], criterion='entropy')

Entropy Score for node {'Manzana': 100, 'Platano': 400, 'Mango': 5, 'Uva': 550} : 1.379
Entropy Score for node {'Manzana': 100, 'Platano': 400, 'Mango': 5} : 0.795
Entropy Score for node {'Platano': 400, 'Mango': 5} : 0.096
Entropy Score for node {'Platano': 400} : 0.0
Information gain: 0.96
